# Imports

In [2]:
import sys
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install scikit-learn
!{sys.executable} -m pip install pillow

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import numpy as np
from scipy.stats import linregress
import tkinter as tk
from PIL import ImageGrab
import turtle
import subprocess

# Draw image

In [4]:
def dump_gui(window, filename = "gui_image_grabbed.png"):
    """
    takes a png screenshot of a tkinter window, and saves it on in cwd
    """
    print('...dumping gui window to png')

    x0 = window.winfo_rootx()
    y0 = window.winfo_rooty()
    x1 = x0 + window.winfo_width()
    y1 = y0 + window.winfo_height()
    ImageGrab.grab().crop((x0, y0, x1, y1)).save(filename)

## Sierpinski Triangle

In [7]:
def draw_sierpinski(turtle, length, depth):
    if depth == 0:
        for i in range(0, 3):
            turtle.fd(length)
            turtle.left(120)
    else:
        draw_sierpinski(turtle, length / 2, depth - 1)
        turtle.fd(length / 2)
        draw_sierpinski(turtle, length / 2, depth - 1)
        turtle.bk(length / 2)
        turtle.left(60)
        turtle.fd(length / 2)
        turtle.right(60)
        draw_sierpinski(turtle, length / 2, depth - 1)
        turtle.left(60)
        turtle.bk(length / 2)
        turtle.right(60)

## Run

In [8]:
root = tk.Tk()
canvas = tk.Canvas(root, width=500, height=500)
canvas.pack()

t = turtle.RawTurtle(canvas)

t.penup()
t.goto(-200, -175)
t.pendown()
draw_sierpinski(t, 400, 3)
t.hideturtle()

dump_gui(root, "Triangle.png")

root.mainloop()

...dumping gui window to png


# Run C++ executable

In [14]:
runTimes = 10
filename = "Triangle.png"

subprocess.call(args=["Fast Box Counting.exe", "-f", f"{filename}", "-t", f"{runTimes}"], stdin=None, stdout=None, stderr=None, shell=False)

0

# Calculate statistics

In [33]:
times = []
with open("results_time.txt", "r") as fin:
    for _ in range(runTimes):
        times.append(float(fin.readline()[:-1]))

for i in range(runTimes):
    sizes = [] # x = s
    boxes = [] # y = n(s)
    with open(f"results_{i}.txt", "r") as fin:
        line = fin.readline()[:-1].split(' ')
        while line != [""]:
            sizes.append(np.log(1 / int(line[0])))
            boxes.append(np.log(int(line[1])))
            
            line = fin.readline()[:-1].split(' ')
    
    print("----------------------------------")
    print(f"Run {i}")
    print(f"Dimension = {linregress(sizes, boxes).slope}")
    print(f"Time spent: {times[i]}")
print(f"Time spent mean: {np.mean(times)}")
print(f"Time spent standard deviation: {np.std(times)}")

----------------------------------
Run 0
Dimension = 1.5003093350307672
Time spent: 0.0004455
----------------------------------
Run 1
Dimension = 1.5003093350307672
Time spent: 0.0004828
----------------------------------
Run 2
Dimension = 1.5003093350307672
Time spent: 0.0005189
----------------------------------
Run 3
Dimension = 1.5003093350307672
Time spent: 0.0006339
----------------------------------
Run 4
Dimension = 1.5003093350307672
Time spent: 0.000674
----------------------------------
Run 5
Dimension = 1.5003093350307672
Time spent: 0.0006796
----------------------------------
Run 6
Dimension = 1.5003093350307672
Time spent: 0.0007197
----------------------------------
Run 7
Dimension = 1.5003093350307672
Time spent: 0.0008994
----------------------------------
Run 8
Dimension = 1.5003093350307672
Time spent: 0.001541
----------------------------------
Run 9
Dimension = 1.5003093350307672
Time spent: 0.0005897
Time spent mean: 0.00071845
Time spent standard deviation: 0.0